**Sequential changepoint detection**

Instructor: Vincent Lostanlen, LS2N, CNRS

Return to: vincent.lostanlen@ls2n.fr


Student name(s): `FILL`

The goal of this assignment is to detect changepoints in audio signals. For this purpose, we will use a Python library named librosa.

To learn more about librosa, visit: https://librosa.org/

In [ ]:
!pip install pandas
!pip install tqdm

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

from IPython.display import Audio, display
import librosa
import librosa.display
import matplotlib
import numpy as np
import pandas as pd
import scipy
import sklearn
from sklearn.metrics import precision_recall_curve
import tqdm

for module in [librosa, matplotlib, np, pd, scipy, sklearn, tqdm]:
    print("{} version: {}".format(module.__name__, module.__version__))

**Part I. Detecting a sine wave in noise**

*Exercise*

Design a real-valued signal of duration equal to two seconds. The first half should contain only white noise of amplitude 1. The second half should contain a mixture of white noise of amplitude 1 and a sine wave of amplitude `a` and frequency `f`.

In [ ]:
def sine_plus_noise(a, f, sr):
    """Return a signal
    x(t) = a sin(2pi f t) + N(t)
    
    where N(t) are i.i.d. samples from the
    standard normal distribution.
    The duration of x is equal to two seconds.
    
    Parameters
    ----------
    a: signal-to-noise ratio
    f: frequency of the wave in Hertz
    decay_time: decay time in seconds
    """
    # TODO
    # Consider using np.arange, np.sin, np.randn
    # signal =
    # noise =
    return (signal+noise)

*Questions*

1. Set `sr=22050 Hz`. How many samples are there in `sine(a, f, sr)`?
2. Does the sign of `a` matter for audio perception? Why?
3. Does the sign of `f` matter for audio perception? Why?

Now let us listen to a sample.

In [ ]:
a = 10.0
f = 500
sr = 22050

x = sine_plus_noise(a, f, sr)
Audio(x, rate=sr)

*Question*

4. Make sure you can hear the sine tone. Then lower the value of `a` until you can no longer hear it (given your current audio setup). What value do you find?

Let us now visualize the waveform before and after the changepoint.

In [ ]:
a = 10
f = 500
sr = 22050

t = np.arange(0, 2, 1/sr)
x = sine_plus_noise(a, f, sr)

plt.figure(figsize=(10, 3))
plt.subplot(1, 2, 1)
plt.plot(t, x)
plt.grid(linestyle="--")
plt.xlim(0.5, 0.51)
plt.subplot(1, 2, 2)
plt.plot(t, x)
plt.xlim(1.5, 1.51)
plt.grid(linestyle="--")

*Question*

5. What do you observe when varying `a` with `f=500 Hz`?
6. Set `a` to `1` and `f` to `11025 Hz`. What happens? Explain.

**Part II. Waveform thresholding**

In [ ]:
def evaluate(feature, a, n_trials, f=500):
    sr = 22050
    Y = []
    for trial in tqdm.tqdm(range(n_trials)):
        x = sine_plus_noise(a, f, sr)
        y = feature(x)
        Y.append(y)
    return np.array(Y)

*Question*

1. What does the `evaluate` function do?
2. What is the type of the first argument, `feature`?

Now, we evaluate a simple-minded feature for event detection, which extract the maximum value of the waveform over the time dimension.

In [ ]:
def max_waveform(x):
    return max(x)

Y0 = evaluate(max_waveform, a=0, n_trials=100)
Y1 = evaluate(max_waveform, a=1, n_trials=100)

plt.hist(Y0, alpha=0.5, label="a = 0");
plt.hist(Y1, alpha=0.5, label="a = 1");
plt.xlim(3.5, 6.5)
plt.grid(linestyle="--")
plt.legend()

*Questions*

3. Why does the output `y = feature(x)` vary between independent trials?

4. Threshold `Y1` and `Y0` with a threshold equal to `delta = 4.5`. How many TP/FP/TN/FN do you obtain?

Now, we plot a precision-recall curve for the `max_waveform` feature.

In [ ]:
Y_pred = np.concatenate([Y0, Y1])
Y_true = np.concatenate([np.zeros(Y0.shape), np.ones(Y1.shape)])
precisions, recalls, thresholds = precision_recall_curve(Y_true, Y_pred)

plt.figure(figsize=(4, 4))
plt.plot(recalls, precisions)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.xlim(0, 1.05)
plt.ylim(0, 1.05)
plt.grid(linestyle="--")

*Question*

5. Which threshold leads to highest precision?

6. Which threshold leads to highest recall?

*Exercises*

a. Write a function `f1_score` which computes the sequence of F1-scores associated to `precisions` and `recalls`.

b. Write a function `best_threshold` which computes the threshold yielding the best F1-score.

c. Write a function `area` which computes the area under the precision-recall curve.

In [ ]:
def f1_score(precisions, recalls):
    """Compute the F1-scores associated to a sequence
    of precisions and recalls.
    
    F = 2*P*R / (P+R)
    
    where P is precision and R is recall.
    
    Parameters
    ----------
    precisions: array of precisions
    recalls: array of recalls
    """
    # TODO
    return f1s


def best_threshold(f1s, thresholds):
    """Find the best threshold associated to a sequence
    of F1-scores.
    
    threshold = argmax_{tau} F(tau) 
    
    where F(tau) is the f1 score for threshold tau.
    
    Parameters
    ----------
    f1s: array of F1-scores
    thresholds: array of thresholds
    """
    # TODO
    # Consider using np.argmax
    threshold = thresholds[np.argmax(f1s)]
    return threshold


def area(precisions, recalls):
    """Compute the area under the precision-recal curve (AUPRC).
    
    Parameters
    ----------
    precisions: array of precisions
    recalls: array of recalls
    """
    # TODO
    # Consider using np.trapz
    # Consider using array[::-1] to reverse an array
    return auprc

*Questions*

7. What is the best threshold and the corresponding F1-score?

8. What is the area under the precision-recall curve (AUPRC) ?

9. What happens to AUPRC if `a=2` instead of `a=1`?

10. What happens to AUPRC if `a=0.5` instead of `a=1`?

*Exercise*

d. Modify the function `evaluate` so that the waveform `x` is multiplied
by a constant `k`.

In [ ]:
def evaluate_k(k, feature, a, n_trials, f=500):
    """
    Compute:
    
    Y(i) = feature(k x_i)
    
    where the signals x_i are independent
    samples of `sine_plus_noise`.
    
    Parameters
    ----------
    k: gain factor
    feature: feature extractor
    a: amplitude
    n_trials: number of signals x_i
    f: sine wave frequency in Hertz
    """
    sr = 22050
    Y = []
    for trial in tqdm.tqdm(range(n_trials)):
        x = k * sine_plus_noise(a, f, sr)
        y = feature(x)
        Y.append(y)
    return np.array(Y)

*Question*

11. Run `evaluate_k` with `k=2`. What is the best threshold and F1-score? What is the AUPRC?

12. Same question with `k=0.5`.

13. Is the `max_waveform` feature invariant or equivariant to multiplication by `k`?

**Part III. Spectral flux**

In this part, we will try to improve the simple-minded feature above (`max_waveform`).

We will design another feature: spectral flux; that is, the rectified temporal first-order difference of the spectrogram, followed by global maximum pooling over the time-frequency domain.

*Exercise*

Write a function `spectral_flux` which extracts:

```
flux = max_{t, f} max(S(t+1, f) - S(t, f), 0)
```

where `S` is the spectrogram of the signal `x`.

In [ ]:
def spectral_flux(x):
    """Spectral flux (linear amplitude scaling).
    
    flux = max_{t, f} max(S(t+1, f) - S(t, f), 0)
    
    Parameters
    ----------
    x: input signal
    """
    S = np.abs(librosa.stft(x))
    # Consider using: np.diff, np.maximum, np.max
    return flux

In [ ]:
Y0 = evaluate(spectral_flux, a=0, n_trials=100)
a = 1
Y1 = evaluate(spectral_flux, a=a, n_trials=100)

plt.hist(Y0, alpha=0.5, label="a = 0");
plt.hist(Y1, alpha=0.5, label="a = {}".format(a));
plt.grid(linestyle="--")
plt.legend()

In [ ]:
Y_pred = np.concatenate([Y0, Y1])
Y_true = np.concatenate([np.zeros(Y0.shape), np.ones(Y1.shape)])
precisions, recalls, thresholds = precision_recall_curve(Y_true, Y_pred)

plt.figure(figsize=(4, 4))
plt.plot(recalls, precisions)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.xlim(0, 1.05)
plt.ylim(0, 1.05)
plt.grid(linestyle="--")

*Questions*
1. Evaluate spectral flux with `a=0` versus `a=1`. What results do you obtain?

2. What is the area under the precision-recall curve (AUPRC) ?

3. What happens to AUPRC if `a=0.5` instead of `a=1`?

4. What happens to AUPRC if `a=0.25` instead of `a=1`?

5. Is the `spectral_flux` feature invariant or equivariant to frequency `f`? Why?

6. Is the `spectral_flux` feature invariant or equivariant to multiplication by `k`? Why?


**Part IV. Logarithmic transformation**

In this part, we will modify the spectral flux so that it computes rectified differences over the *logarithms* of spectrogram magnitudes.

*Exercise*


Write a function `decibel_flux` which extracts:

```
flux = max_{t, f} max(10 log_10 S(t+1, f) - 10 log_10 S(t, f), 0)
```

where `S` is the magnitude spectrogram of the signal `x`.

In [ ]:
def decibel_flux(x):
    """Spectral flux (logarithmic amplitude scaling).
    
    flux = max_{t, f} max(10 log_10 S(t+1, f) - 10 log_10 S(t, f), 0)
    
    Parameters
    ----------
    x: input signal
    """
    S = np.abs(librosa.stft(x))
    # Consider using: np.log10, np.diff, np.maximum, np.max
    return flux

In [ ]:
x = sine_plus_noise(a=1, f=500, sr=22050)

fig, ax = plt.subplots(1, 3, figsize=(9, 3), sharey=True)
for i, feature in enumerate([max_waveform, spectral_flux, decibel_flux]):
    ax[i].plot(range(1, 10), [feature(k*x)/feature(x) for k in range(1, 10)], "-o")
    ax[i].set_ylim(0, 10)
    ax[i].set_ylim(0, 10)
    ax[i].grid(linestyle="--")
    ax[i].set_xlabel("Multiplicative factor k")
    ax[i].set_ylabel("feature(kx)")
    ax[i].set_title(feature.__name__)
plt.tight_layout()

1. Is the `decibel_flux` feature invariant or equivariant to frequency `f`? Why?

2. Is the `decibel_flux` feature invariant or equivariant to multiplication by `k`? Why?

In [ ]:
Y0 = evaluate(decibel_flux, a=0, n_trials=100)
a = 1
Y1 = evaluate(decibel_flux, a=a, n_trials=100)

plt.hist(Y0, alpha=0.5, label="a = 0");
plt.hist(Y1, alpha=0.5, label="a = {}".format(a));
plt.grid(linestyle="--")
plt.legend()

In [ ]:
Y_pred = np.concatenate([Y0, Y1])
Y_true = np.concatenate([np.zeros(Y0.shape), np.ones(Y1.shape)])
precisions, recalls, thresholds = precision_recall_curve(Y_true, Y_pred)

plt.figure(figsize=(4, 4))
plt.plot(recalls, precisions)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.xlim(0, 1.05)
plt.ylim(0, 1.05)
plt.grid(linestyle="--")

area(precisions, recalls)

*Questions*

1. Evaluate spectral flux with `a=0` versus `a=1`. What results do you obtain?

2. What is the area under the precision-recall curve (AUPRC) ?

3. Raise `a` until the AUPRC is at least `0.9`. What value of `a` do you get?

4. Explain why `decibel_flux` is worse than `spectral_flux` in low-SNR (low `a`) settings.

**Part V. Per-Channel Energy Normalization**

In this section, we will apply per-channel energy normalization (PCEN) onto the spectrogram as an alternative to decibel-scaled spectral flux.

*Exercise*

Write a function `max_pcen` which extracts

```
maxP = max_{t,f} E[t, f] / M[t, f]
```

where `E` is the magnitude scalogram of `x` and
```
M[t, f] = 1/(1+t) \sum_{tau<=t} E[tau, f]
```

is the cumulative mean of `E` over the time dimension.

Try vectorizing your code! Use `np.cumsum` instead of a `for` loop.

In [ ]:
def max_pcen(x):
    """Maximum-pooled PCEN.
    
    maxP = max_{t,f} E[t, f] / M[t, f]
    
    Parameters
    ----------
    x: input signal
    """
    E = np.abs(librosa.stft(x))
    # Consider using: np.arange, np.cumsum, np.newaxis, np.max
    return maxP

*Questions*

1. Is the `max_pcen` feature invariant or equivariant to frequency `f`? Why?

2. Is the `max_pcen` feature invariant or equivariant to multiplication by `k`? Why?

In [ ]:
Y0 = evaluate(max_pcen, a=0, n_trials=100)
a = 1
Y1 = evaluate(max_pcen, a=a, n_trials=100)

plt.hist(Y0, alpha=0.5, label="a = 0");
plt.hist(Y1, alpha=0.5, label="a = 1".format(a));
plt.grid(linestyle="--")
plt.legend()

*Questions*

3. Evaluate PCEN with `a=0` versus `a=1`. What results do you obtain?

4. Lower `a` until the AUPRC is below `0.9`. What value of `a` do you get?

5. What is the resemblance between `pcen` and `decibel_flux`? What is the difference?

**Part VI. Analysis of variance**

In this, section, we will perform an analysis of variance (ANOVA) between two groups: before and after the 

*Exercise*

Write a function `anova(x)` which extracts

```
maxR = max_{t,f} V[f] / (V_past[t, f] + V_future[t, f])
```

where `V = (\sum_{t,f} E[t,f]**2) - (\sum_{t,f} E[t,f])**2` is the global per-channel variance of `E` and `V_past[t,f]` (resp. `V_future[t,f]`) are the per-channel variances before (resp. after) a candidate changepoint `t`.

Try vectorizing your code! Use `np.cumsum` instead of a `for` loop.

In [ ]:
def anova(x):
    """Analysis of Variance (ANOVA).
    
    maxR = max_{t,f} V[f] / (V_past[t, f] + V_future[t, f])
    
    Parameters
    ----------
    x: input signal
    """
    E = np.abs(librosa.stft(x))
    # Consider using:
    # np.arange, np.cumsum, np.newaxis, np.var, np.max
    return max_ratio

In [ ]:
Y0 = evaluate(anova, a=0, n_trials=100)
a = 1.0
Y1 = evaluate(anova, a=a, n_trials=100)

plt.hist(Y0, alpha=0.5, label="a = 0");
plt.hist(Y1, alpha=0.5, label="a = {}".format(a));
plt.grid(linestyle="--")
plt.legend()

*Questions*

1. Is the `anova` feature invariant or equivariant to frequency `f`? Why?

2. Is the `anova` feature invariant or equivariant to multiplication by `k`? Why?

3. Evaluate `anova` with `a=0` versus `a=1`. What results do you obtain?

4. Lower `a` until the AUPRC is below `0.9`. What value of `a` do you get?

5. Is there a drawback of using `anova` versus all other proposed methods?

**Part VI. Benchmark**

In [ ]:
sr = 22050
n_trials = 100
f = 500
amplitudes = [-20, -15, -10] + list(range(-10, 1)) + [3, 6, 9, 12, 15, 20, 25, 30]
AUPRC = {}
features = [max_waveform, spectral_flux, decibel_flux, max_pcen, anova]
Y0 = {feature.__name__: [] for feature in features}

for trial in tqdm.tqdm(range(n_trials)):
    x = sine_plus_noise(a=0, f=f, sr=sr)
    for feature in features:
        y = feature(x)
        Y0[feature.__name__].append(y)
    
for feature in features:
    Y0[feature.__name__] = np.array(Y0[feature.__name__])

for a_dB in tqdm.tqdm(amplitudes):
    a = 10**(a_dB/10)
    for feature in features:
        AUPRC[(a_dB, feature.__name__)] = []
    Y1 = {feature.__name__: [] for feature in features}
    for trial in range(n_trials):
        x = sine_plus_noise(a=a, f=f, sr=sr)
        for feature in features:
            Y1[feature.__name__].append(feature(x))
    for feature in features:
        Y1[feature.__name__] = np.array(Y1[feature.__name__])
        Y_pred = np.concatenate([Y0[feature.__name__], Y1[feature.__name__]])
        Y_true = np.concatenate([
            np.zeros(Y0[feature.__name__].shape), np.ones(Y1[feature.__name__].shape)])
        precisions, recalls, thresholds = precision_recall_curve(Y_true, Y_pred)
        AUPRC[(a_dB, feature.__name__)].append(area(precisions, recalls))

In [ ]:
plt.figure(figsize=(6, 5))
for feature in features:
    plt.plot(amplitudes, [AUPRC[a, feature.__name__] for a in amplitudes], "-o", label=feature.__name__)
plt.legend()
plt.gca().invert_xaxis()
plt.grid(linestyle="--")
plt.xlabel("Signal-to-noise ratio (SNR)")
plt.ylabel("AUPRC")
plt.title("Changepoint detection benchmark")

*Questions*

1. Execute the code above. What does it do? Which system is the best?

2. Rate systems from best to worst in terms of robustness to noise.

3. Which of these systems invariant to multiplication by `k`?

4. Which are spectrogram-based?

5. Which take temporal context into account beyond one spectrogram frame?

6. Which are causal?